In [1]:
from datetime import datetime
from core.analysis import analyze_stock_no_parse
from maintenance.manual import batch_analyze_ranked_stock
from services.ai_service import ai_search
import logging
import json
from services.tushare import find_stock_code
from utils.common import time_now

log = logging.getLogger(__name__)
log.info('imports loaded')

2025-02-19 20:00:22 INFO [243342219.py]: imports loaded


In [ ]:
def ai_search_workflow(query):
    res_json = ai_search(query)
    time = datetime.now()
    time_str = time.strftime('%m%d_%H%M')
    filename = f"{query[:8]}_{time_str}.md"
    save_path = f"/Users/cgl/Library/Mobile Documents/iCloud~md~obsidian/Documents/md/ask_ai/ai_search/{filename}"
    references_path =  f"/Users/cgl/Library/Mobile Documents/iCloud~md~obsidian/Documents/md/ask_ai/ai_search/references/{filename}"
    references_content = ''
    save_content = f"**问题：** {res_json['query']}\n\n"
    save_content += f"{res_json['answer']}"
    search_result = res_json['search_result']
    log.info(search_result)
    references_content = ''
    for s in search_result:
           references_content += f"### 搜索词：{s['keyword']}\n\n"
           for r in s['results']:
                references_content += f"标题: {r['title']}\n\n"
                references_content += f"摘要文本: {r['snippet']}\n\n"
                references_content += f"来源链接: {r['link']}\n\n"
                references_content += f"内容: {r['content']}\n\n"
    with open(save_path, "w") as f:
            f.write(save_content)
    with open(references_path, "w") as f:
            f.write(references_content)
    log.info("AI搜索分析结束，文件已保存")
log.info('ai_search_workflow loaded')

# ai搜索
query="""分析一下深桑达A这只股票今天的表现，并预测一下明日的走势"""
ai_search_workflow(query)

2025-02-24 16:46:06 INFO [1783528401.py]: ai_search_workflow loaded
2025-02-24 16:46:08 INFO [_client.py]: HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-24 16:46:08 INFO [ai_service.py]: 搜索关键词： 万丰奥威 公司介绍 主营业务
2025-02-24 16:46:09 INFO [lib.rs]: response: https://lite.duckduckgo.com/lite/ 200 19466
2025-02-24 16:46:10 INFO [ai_service.py]: 搜索关键词： 万丰奥威 涨停原因 2025年2月24日
2025-02-24 16:46:11 INFO [lib.rs]: response: https://html.duckduckgo.com/html 200 26203
2025-02-24 16:48:30 INFO [_client.py]: HTTP Request: POST https://api.siliconflow.cn/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-24 16:48:30 INFO [1783528401.py]: [{'index': 0, 'keyword': '万丰奥威 公司介绍 主营业务', 'results': [{'snippet': '1、公司简介基本情况：浙江 万丰奥威 汽轮股份有限公司（002085.sz）是一家以先进制造业为核心的国际化公司，2006年深交所上市。2020年，公司正式确立以"镁合金-铝合金-轻质高强度钢"金属材料轻量化应用为主线的新发展格局，实施"双引擎 "驱动战略：成为"全球汽车金属部件轻量化推动者...', 'title': '1分钟了解上市企业——万丰奥威 1、公司简介基本情况：浙江 万丰奥威 汽轮股份有限公司（002085.sz）是一家以先进制造业为核心的国际化 ...', 'link'

In [ ]:
def ai_stock_analysis_workflow(ts_code: str=None, name: str=None):
    """
    根据技术面和消息面分析股票，需传入ts_code或者name股票名称至少一个。
    有限根据ts_code
    """
    if ts_code is None:
        if name is None:
            raise ValueError("ts_code和name必须传入一个")
        ts_code = find_stock_code(name)
        if ts_code is None:
            raise ValueError("输入的股票名称不正确")
    prompt, suggestion=analyze_stock_no_parse(ts_code)
    filename = f"stock_{ts_code[:6]}_{time_now('%m%d_%H%M')}.md"
    save_path = f"/Users/cgl/Library/Mobile Documents/iCloud~md~obsidian/Documents/md/ask_ai/stock_analyze/{filename}"
    save_content = f"# 个股分析：{ts_code}"
    save_content += '\n\n'
    save_content += suggestion
    save_content += '\n\n**提问词：**\n'
    save_content += prompt
    with open(save_path, "w") as f:
         f.write(save_content)

# 结合tushare股票最近股价数据和相关新闻快讯，分析股票短中长期投资建议
ai_stock_analysis_workflow(name='电魂网络')

In [ ]:
def loop_analyze():
    for i  in range(5):
        batch_analyze_ranked_stock()